In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix, classification_report
import torchvision.models as models
import torch.nn as nn
import torch

In [5]:
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD  = [0.229, 0.224, 0.225]

IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2

In [8]:
test_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),                       
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD)              
])


dataset_path = '../../../dataset/test'
test_dataset   = datasets.ImageFolder(
    root=dataset_path, transform=test_transforms,
)

test_loader = DataLoader(
    test_dataset,
    shuffle=False,
    pin_memory=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS, 
)



In [9]:
def test_model(model, loader, criterion, device):
    model.eval()
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [10]:
model = models.mobilenet_v2(weights=None)

trained_model_path = 'best_mobilenetv2_teadiseases.pth'
model.load_state_dict(torch.load(trained_model_path))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()

test_loss, test_acc = test_model(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Test Loss: 0.3361, Test Accuracy: 0.8844


In [11]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.numpy())

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n",
      classification_report(all_labels, all_preds, target_names=test_dataset.classes))

Confusion Matrix:
 [[160   0   3   1   1   4   0   1   0]
 [  2 115  12   1   1   1   1   1   0]
 [  6  24 122   1   6   3   0   1   0]
 [  0   0   0 134  15   0   0   1   0]
 [  0   0   0   9 141   0   0   0   0]
 [  2   0   2   0   0  20   0   0   0]
 [  0   0   0   0   0   1  20   0   0]
 [  2   6   0   5   2   0   0 157   0]
 [  0   0   0   0   0   0   0   0  11]]

Classification Report:
                      precision    recall  f1-score   support

         algal_spot       0.93      0.94      0.94       170
       brown_blight       0.79      0.86      0.82       134
        gray_blight       0.88      0.75      0.81       163
            healthy       0.89      0.89      0.89       150
         helopeltis       0.85      0.94      0.89       150
           red-rust       0.69      0.83      0.75        24
red-spider-infested       0.95      0.95      0.95        21
           red_spot       0.98      0.91      0.94       172
         white-spot       1.00      1.00      1.00    